# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [84]:
!wget https://github.com/PiginIvan/Archives/raw/main/ds_salaries.csv

--2024-06-11 10:58:27--  https://github.com/PiginIvan/Archives/raw/main/ds_salaries.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PiginIvan/Archives/main/ds_salaries.csv [following]
--2024-06-11 10:58:27--  https://raw.githubusercontent.com/PiginIvan/Archives/main/ds_salaries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210076 (205K) [text/plain]
Saving to: ‘ds_salaries.csv.1’

ds_salaries.csv.1   100%[===================>] 205.15K  --.-KB/s    in 0.1s    

2024-06-11 10:58:28 (1.63 MB/s) - ‘ds_salaries.csv.1’ saved [210076/210076]



In [85]:
import pandas as pd
import numpy as np

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [144]:
from sklearn.model_selection import train_test_split

y = df['salary_in_usd']
X = df.drop(columns=['salary_in_usd', 'salary'])
# Так как мы хотим предсказывать зарплату, а делаем это по столбцу 'salary_in_usd', то столбец 'salary' можем сильно нарушать работу
# нашей модели, так как напрямую коррелирует с целевой переменной.

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

print(f"Доля выборки от исходного датасета: обучающая-обучающая: {X_train.shape[0] / X.shape[0]}")
print(f"Доля выборки от исходного датасета: тестовая: {X_test.shape[0] / X.shape[0]}")
print(f"Доля выборки от исходного датасета: обучающая-валидационная: {X_val.shape[0] / X.shape[0]}")

Доля выборки от исходного датасета: обучающая-обучающая: 0.8098535286284954
Доля выборки от исходного датасета: тестовая: 0.10013315579227697
Доля выборки от исходного датасета: обучающая-валидационная: 0.09001331557922769


## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [145]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

categorical_features = (X.dtypes == "object").values

X_real_valued_train = X_train[X_train.columns[~categorical_features]]
X_real_valued_test = X_test[X_test.columns[~categorical_features]]
X_real_valued_val = X_val[X_val.columns[~categorical_features]]
X_categorical_train = X_train[X_train.columns[categorical_features]]
X_categorical_test = X_test[X_test.columns[categorical_features]]
X_categorical_val = X_val[X_val.columns[categorical_features]]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_encoded_train = pd.DataFrame(data=encoder.fit_transform(X_categorical_train), columns=encoder.get_feature_names_out(), index=X_train.index)
X_encoded_test = pd.DataFrame(data=encoder.transform(X_categorical_test), columns=encoder.get_feature_names_out(), index=X_test.index)
X_encoded_val = pd.DataFrame(data=encoder.transform(X_categorical_val), columns=encoder.get_feature_names_out(), index=X_val.index)

X_train = pd.concat([X_real_valued_train, X_encoded_train], axis=1)
X_test = pd.concat([X_real_valued_test, X_encoded_test], axis=1)
X_val = pd.concat([X_real_valued_val, X_encoded_val], axis=1)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('MAPE:', mean_absolute_percentage_error(y_pred, y_test))
print('RMSE:', np.sqrt(mean_squared_error(y_pred, y_test)))

MAPE: 0.3533399081767709
RMSE: 4790092473274.407


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [147]:
from xgboost.sklearn import XGBRegressor
import numpy as np

min_MAPE = 1e9
best_depth, best_learning_rate, best_n_estimators, best_gamma = 0, 0, 0, 0
depth_values, learning_rate_values = np.arange(3, 15), np.arange(0.01, 0.1, 0.02),
n_estimators_values, gamma_values = np.arange(80, 120, 10), np.arange(0, 0.5, 0.1)

for cur_depth in depth_values:
    for cur_learning_rate in learning_rate_values:
          XGB = XGBRegressor(max_depth=cur_depth, learning_rate=cur_learning_rate, random_state=42)
          XGB.fit(X_train, y_train)
          cur_MAPE = mean_absolute_percentage_error(y_val, XGB.predict(X_val))
          if min_MAPE > cur_MAPE:
            best_depth = cur_depth; best_learning_rate = np.round(cur_learning_rate, 2)
            min_MAPE = cur_MAPE

for cur_n_estimators in n_estimators_values:
    for cur_gamma in gamma_values:
          XGB = XGBRegressor(max_depth=best_depth, learning_rate=best_learning_rate, n_estimators=cur_n_estimators, gamma=cur_gamma, random_state=42)
          XGB.fit(X_train, y_train)
          cur_MAPE = mean_absolute_percentage_error(y_val, XGB.predict(X_val))
          if min_MAPE > cur_MAPE:
            best_n_estimators = cur_n_estimators; best_gamma = cur_gamma
            min_MAPE = cur_MAPE

print(f"Оптимальная глубина: {best_depth}, оптимальная скорость обучения: {best_learning_rate}")
print(f"Оптимальное число базовых моделей: {best_n_estimators}, оптимальное ограничение на разделение: {best_gamma}")

Оптимальная глубина: 14, оптимальная скорость обучения: 0.09
Оптимальное число базовых моделей: 110, оптимальное ограничение на разделение: 0.0


In [148]:
params = {
    'max_depth' : best_depth,
    'learning_rate' : best_learning_rate,
    'n_estimators' : best_n_estimators,
    'gamma' : best_gamma,
}

XGB = XGBRegressor(max_depth=params['max_depth'], learning_rate=params['learning_rate'],
                   n_estimators=params['n_estimators'], gamma=params['gamma'], random_state=666)

In [149]:
%%time
XGB.fit(X_train, y_train)

CPU times: user 1.25 s, sys: 12.9 ms, total: 1.26 s
Wall time: 655 ms


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.09, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=14, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=110, n_jobs=None,
             num_parallel_tree=None, random_state=666, ...)

In [150]:
%%time
pred = XGB.predict(X_test)

CPU times: user 36.8 ms, sys: 65 µs, total: 36.8 ms
Wall time: 36.3 ms


In [163]:
print(f'MAPE: {mean_absolute_percentage_error(y_test, pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, pred))}')

MAPE: 0.4657176779600158
RMSE: 51177.94628021327


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [93]:
!pip install catboost

In [153]:
from catboost import CatBoostRegressor

min_MAPE = 1e9
best_depth, best_learning_rate, best_iterations = 0, 0, 0,
depth_values, learning_rate_values = np.arange(3, 15), np.arange(0.01, 0.1, 0.02)
iterations_values = np.arange(10, 50, 10)

for cur_depth in depth_values:
    for cur_learning_rate in learning_rate_values:
      for cur_iterations in iterations_values:
          CTR = CatBoostRegressor(max_depth=cur_depth, learning_rate=cur_learning_rate, iterations=cur_iterations, verbose=False, random_state=42)
          CTR.fit(X_train, y_train)
          cur_MAPE = mean_absolute_percentage_error(y_val, CTR.predict(X_val))
          if min_MAPE > cur_MAPE:
            best_depth = cur_depth; best_learning_rate = np.round(cur_learning_rate, 2); best_iterations = cur_iterations
            min_MAPE = cur_MAPE

print(f"Оптимальная глубина: {best_depth}, оптимальная скорость обучения: {best_learning_rate}")
print(f"Оптимальное число итераций: {best_iterations}")

Оптимальная глубина: 11, оптимальная скорость обучения: 0.09
Оптимальное число итераций: 40


In [154]:
params = {
    'depth' : best_depth,
    'learning_rate' : best_learning_rate,
    'iterations' : best_iterations
}

CTR = CatBoostRegressor(max_depth=params['depth'], learning_rate=params['learning_rate'], iterations=params['iterations'], verbose=False, random_state=42)

In [164]:
%%time
CTR.fit(X_train, y_train)

CPU times: user 726 ms, sys: 42.9 ms, total: 769 ms
Wall time: 459 ms


In [165]:
%%time
pred = CTR.predict(X_test)

CPU times: user 10.7 ms, sys: 988 µs, total: 11.7 ms
Wall time: 10.5 ms


In [166]:
print(f'MAPE: {mean_absolute_percentage_error(y_test, pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, pred))}')

MAPE: 0.4657176779600158
RMSE: 51177.94628021327


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [167]:
from catboost import Pool

y = df['salary_in_usd']
X = df.drop(columns='salary_in_usd')
X.drop(columns='salary', inplace=True)

X_train_full_POOL, X_test_POOL, y_train_full_POOL, y_test_POOL = train_test_split(X, y, test_size=0.1, random_state=42)
X_train_POOL, X_val_POOL, y_train_POOL, y_val_POOL = train_test_split(X_train_full_POOL, y_train_full_POOL, test_size=0.1, random_state=42)

categorical_features = (X.dtypes == "object").values
categorical_indices = np.where(categorical_features == True)[0]

X_train_POOL = Pool(data=X_train_POOL, label=y_train_POOL, cat_features=categorical_indices)
X_test_POOL = Pool(data=X_test_POOL, label=y_test_POOL, cat_features=categorical_indices)
X_val_POOL = Pool(data=X_val_POOL, label=y_val_POOL, cat_features=categorical_indices)

In [169]:
from catboost import CatBoostRegressor

min_MAPE = 1e9
best_depth, best_learning_rate, best_iterations = 0, 0, 0,
depth_values, learning_rate_values = np.arange(3, 15), np.arange(0.01, 0.1, 0.02)
iterations_values = np.arange(10, 60, 10)

for cur_depth in depth_values:
    for cur_learning_rate in learning_rate_values:
      for cur_iterations in iterations_values:
          CTR = CatBoostRegressor(max_depth=cur_depth, learning_rate=cur_learning_rate, iterations=cur_iterations, verbose=False, random_state=42)
          CTR.fit(X_train_POOL)
          cur_MAPE = mean_absolute_percentage_error(y_val_POOL, CTR.predict(X_val_POOL))
          cur_RMSE = np.sqrt(mean_squared_error(y_val_POOL, CTR.predict(X_val_POOL)))
          if min_MAPE > cur_MAPE:
            best_depth = cur_depth; best_learning_rate = np.round(cur_learning_rate, 2); best_iterations = cur_iterations
            min_MAPE = cur_MAPE

print(f"Оптимальная глубина: {best_depth}, оптимальная скорость обучения: {best_learning_rate}")
print(f"Оптимальное число итераций: {best_iterations}")

Оптимальная глубина: 7, оптимальная скорость обучения: 0.09
Оптимальное число итераций: 50


In [170]:
params = {
    'depth' : best_depth,
    'learning_rate' : best_learning_rate,
    'iterations' : best_iterations
}

CTR = CatBoostRegressor(max_depth=params['depth'], learning_rate=params['learning_rate'], iterations=params['iterations'], verbose=False, random_state=42)

In [171]:
%%time
CTR.fit(X_train_POOL)

CPU times: user 321 ms, sys: 72 ms, total: 392 ms
Wall time: 385 ms


In [172]:
%%time
pred = CTR.predict(X_test_POOL)

CPU times: user 2.26 ms, sys: 3 µs, total: 2.27 ms
Wall time: 1.26 ms


In [173]:
print(f'MAPE: {mean_absolute_percentage_error(y_test_POOL, pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test_POOL, pred))}')

MAPE: 0.4855301804220188
RMSE: 51321.10268993739


**Ответ:** **качество по метрикам MAPE и RMSE модели стали хуже с использованием Pool, соответвенно это не имеет смысла.**

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [175]:
from lightgbm import LGBMRegressor

min_MAPE = 1e9
best_depth, best_learning_rate, best_n_estimators = 0, 0, 0,
depth_values, learning_rate_values = np.arange(3, 15), np.arange(0.01, 0.1, 0.02)
n_estimators_values = np.arange(80, 120)

for cur_depth in depth_values:
    for cur_learning_rate in learning_rate_values:
      for cur_n_estimators in n_estimators_values:
          LGBM = LGBMRegressor(max_depth=cur_depth, learning_rate=cur_learning_rate, n_estimators=cur_n_estimators, verbose=-1, verbose_eval=False, random_state=42)
          LGBM.fit(X_train, y_train)
          cur_MAPE = mean_absolute_percentage_error(y_val, LGBM.predict(X_val))
          if min_MAPE > cur_MAPE:
            best_depth = cur_depth; best_learning_rate = np.round(cur_learning_rate, 2); best_n_estimators = cur_n_estimators
            min_MAPE = cur_MAPE

print(f"Оптимальная глубина: {best_depth}, оптимальная скорость обучения: {best_learning_rate}")
print(f"Оптимальное число итераций: {best_n_estimators}")

Оптимальная глубина: 6, оптимальная скорость обучения: 0.09
Оптимальное число итераций: 111


In [176]:
params = {
    'max_depth' : best_depth,
    'learning_rate' : best_learning_rate,
    'n_estimators' : best_n_estimators
}

LGBM = LGBMRegressor(max_depth=params['max_depth'], learning_rate=params['learning_rate'], n_estimators=params['n_estimators'], verbose=-1, random_state=42)

In [177]:
%%time
LGBM.fit(X_train, y_train)

CPU times: user 92.5 ms, sys: 1.98 ms, total: 94.5 ms
Wall time: 211 ms


LGBMRegressor(learning_rate=0.09, max_depth=6, n_estimators=111,
              random_state=42, verbose=-1)

In [178]:
%%time
pred = LGBM.predict(X_test)

CPU times: user 8.37 ms, sys: 1e+03 ns, total: 8.37 ms
Wall time: 8.26 ms


In [179]:
print(f'MAPE: {mean_absolute_percentage_error(y_test, pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, pred))}')

MAPE: 0.38301405756596274
RMSE: 50728.41841601955


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

**Ответ:**

---
Качество:
1. XGboost:
MAPE: 0.36718735880593073 RMSE: 49664.30911433854
2. CatBoost without Pool:
MAPE: 0.4657176779600158 RMSE: 51177.94628021327
3. CatBoost with Pool:
MAPE: 0.5016080849605827 RMSE: 51210.25954987478
4. LightGBM
MAPE: 0.38301405756596274 RMSE: 50728.41841601955
---
Сравнение: наилучший результат по метрикам MAPE и RMSE показала модель XGboost. CatBoost with Pool показала значения хуже, чем CatBoost without Pool, получается использование дополнительного функционала ухудшило результаты. LightGBM показала достаточно близкие к XGboost показатели по MAPE и RMSE, однако немного более слабые. Соответственно наилучшей моделью по качеству можно выделить XGboost, возможно при более полном рассмотрении различных гиперпараметров другие модели могут достичь такого же результата.

---
Скорость обучения:
1. XGboost:
Wall time: 655 ms
2. CatBoost without Pool:
Wall time: 459 ms
3. CatBoost with Pool:
Wall time: 385 ms
4. LightGBM:
Wall time: 211 ms
---
Сравнение: самой быстрой по обучению оказалась LightGBM, самой медленной - XGboost, а CatBoost without Pool и CatBoost with Pool примерно одинаковы и имеют средние значения. Если необходимо будет часто делать обучение моделей, и нам будет важно время исполнения то стоит использовать LightGBM.

---
Скорость Предсказания:
1. XGboost:
Wall time: 36.3 ms
2. CatBoost without Pool:
Wall time: 10.5 ms
3. CatBoost with Pool:
Wall time: 1.26 ms
4. LightGBM:
Wall time: 8.26 ms
---
Сравнение: самой быстрой по предсказанию оказалась CatBoost with Pool, самой медленной - XGboost, LightGBM и Cat Boost without Pool делают предсказания со средним временем между самый быстрой и самой медленной библиотеками. Если необходимо будет часто делать предсказания, и нам важно время исполнения, то стоит использовать CatBoost with Pool.

---
Гиперпараметры:
1. XGboost:
Оптимальная глубина: 14, оптимальная скорость обучения: 0.09
Оптимальное число базовых моделей: 110, оптимальное ограничение на разделение: 0.0
2. CatBoost without Pool:
Оптимальная глубина: 11, оптимальная скорость обучения: 0.09
Оптимальное число итераций: 40
3. CatBoost with Pool:
Оптимальная глубина: 7, оптимальная скорость обучения: 0.09
Оптимальное число итераций: 50
4. LightGBM:
Оптимальная глубина: 6, оптимальная скорость обучения: 0.09
Оптимальное число итераций: 111
---
Сравнение: максимальная глубина деревьев оказалась наибольшей у XGboost(14), а минимальной у LightGBM(6), у CatBoost without Pool среднее между ними(11), а CatBoost with Pool очень близка в этом показателе к LightGBM(7). Скорость обучения не менялась у всех различных моделей, она оставалась одинаковой(0.09). Оптимальное число базовых моделей определялось только для Xgboost, оно равно(110), для остальных моделей можно сравнить оптимальное число итераций. Минимально оно для CatBoost without Pool(40), потом больше для CatBoost with Pool(50) и максимально для LightGBM(111).

# Часть 2 Кластеризация (5 баллов) (ДОПОЛНИТЕЛЬНО)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [180]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [181]:
ratings = ratings.transpose()
ratings.head(5)

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
user,0.000000,1.000000,2.000000,3.0,4.000000,5.000000,6.0,7.0,8.000000,9.000000,...,4990.000000,4991.0,4992.000000,4993.000000,4994.000000,4995.000000,4996.0,4997.000000,4998.0,4999.000000
the beatles,NaN,NaN,NaN,NaN,0.043529,NaN,NaN,NaN,0.093398,0.017621,...,NaN,NaN,0.121169,0.038168,0.007939,0.017884,NaN,0.076923,NaN,NaN
radiohead,0.020417,0.184962,NaN,NaN,0.086281,0.006322,NaN,NaN,NaN,0.019156,...,0.017735,NaN,NaN,NaN,0.011187,NaN,NaN,NaN,NaN,NaN
deathcab for cutie,NaN,0.024561,0.028635,NaN,0.034590,NaN,NaN,NaN,NaN,0.013349,...,0.121344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027893
coldplay,NaN,NaN,NaN,NaN,0.016712,NaN,NaN,NaN,NaN,NaN,...,0.217175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Выкиньте строку под названием `user`.

In [182]:
ratings = ratings.drop('user')
ratings.head(5)

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
the beatles,NaN,NaN,NaN,NaN,0.043529,NaN,NaN,NaN,0.093398,0.017621,...,NaN,NaN,0.121169,0.038168,0.007939,0.017884,NaN,0.076923,NaN,NaN
radiohead,0.020417,0.184962,NaN,NaN,0.086281,0.006322,NaN,NaN,NaN,0.019156,...,0.017735,NaN,NaN,NaN,0.011187,NaN,NaN,NaN,NaN,NaN
deathcab for cutie,NaN,0.024561,0.028635,NaN,0.034590,NaN,NaN,NaN,NaN,0.013349,...,0.121344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027893
coldplay,NaN,NaN,NaN,NaN,0.016712,NaN,NaN,NaN,NaN,NaN,...,0.217175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
modest mouse,NaN,NaN,NaN,NaN,0.015935,NaN,NaN,NaN,NaN,0.030437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [183]:
ratings = ratings.fillna(0)
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
capn jazz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [184]:
from sklearn.cluster import KMeans

Kmeans_cluster = KMeans(n_clusters=5, random_state=42)
Kmeans_cluster.fit(ratings)
ratings['cluster_label'] = Kmeans_cluster.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [185]:
pd.value_counts(ratings['cluster_label'])

cluster_label
2    995
0      2
3      1
4      1
1      1
Name: count, dtype: int64

**Ответ:** **кластеризация получилось не особо полезной, так как практически все объекты изначального датасета были отнесены к одному кластеру. Это могло произойти из-за того, что было выбрано не очень большое число кластеров, а также из-за того, что данные в датасете довольно похожи между собой.**


## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [186]:
# так как кластеры размером 1 имеют индексы 1, 3, 4, то используем их

small_clusters = ratings[ratings['cluster_label'].isin([1, 3, 4])]
pd.DataFrame(small_clusters['cluster_label'])

,cluster_label
the beatles,3
coldplay,4
보아,1


Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [187]:
average_listening_share = pd.DataFrame(ratings.sum(axis=1))
average_listening_share.sort_values(by=0, ascending=False)
average_listening_share

,0
the beatles,94.844883
radiohead,61.256073
deathcab for cutie,34.716688
coldplay,34.151904
modest mouse,31.381284
...,...
michal w. smith,6.476960
群星,4.594432
agalloch,6.987179
meshuggah,4.154773


**Ответ:** **изучив среднюю долю прослушивания для каждого исполнителя и отсортировав доли по убыванию можно увидеть, что `the beatles` является самой прослушиваемой и опережает почти все остальные в несколько раз, поэтому и отделяется в отдельный кластер.**



## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [188]:
from sklearn.preprocessing import normalize

names = ratings.select_dtypes(include=np.number).columns.tolist()

ratings_normed = pd.DataFrame(normalize(ratings), columns=names, index=ratings.index)
ratings_normed.head()

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,cluster_label
the beatles,0.000000,0.000000,0.00000,0.0,0.009272,0.000000,0.0,0.0,0.019894,0.003753,...,0.0,0.02581,0.00813,0.001691,0.003809,0.0,0.016385,0.0,0.000000,0.639016
radiohead,0.006894,0.062457,0.00000,0.0,0.029135,0.002135,0.0,0.0,0.000000,0.006469,...,0.0,0.00000,0.00000,0.003778,0.000000,0.0,0.000000,0.0,0.000000,0.675350
deathcab for cutie,0.000000,0.010010,0.01167,0.0,0.014097,0.000000,0.0,0.0,0.000000,0.005440,...,0.0,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.011368,0.815088
coldplay,0.000000,0.000000,0.00000,0.0,0.003911,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.936205
modest mouse,0.000000,0.000000,0.00000,0.0,0.006293,0.000000,0.0,0.0,0.000000,0.012020,...,0.0,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.789817


Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [198]:
kmeans = KMeans(n_clusters=5, random_state=42)
ratings_normed.columns = ratings_normed.columns.astype(str)
kmeans.fit(ratings_normed)
Kmeans_labels = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [199]:
unique, counts = np.unique(Kmeans_labels, return_counts=True)
counts

array([  5, 852, 141,   1,   1])

**Ответ** **размеры кластеров изменились не столь сильно, кластеризация стала лучше, но все равно несколько кластеров имеют слишком маленький размер, а несколько слишком большой.**

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --